Perfect 👍 Let’s pick a **very easy Digit DP problem** so the pattern becomes clear.

---

### Problem:

👉 Count how many numbers ≤ `N` do **not contain the digit 7**.

This is simple and helps you understand `tight`, `leading_zero`, and transitions.

---

### Step 1: Break into digits

Suppose `N = 325`.
Digits = `[3, 2, 5]`.
We’ll process left to right.

---

### Step 2: State Definition

We define our DP state as:

```
dp[pos][tight][leading_zero] = number of valid numbers from pos → end
```

* `pos`: current digit index we are processing.
* `tight`: whether we are bound by N’s prefix (`1` = yes, `0` = free).
* `leading_zero`: whether we are still placing leading zeros.

---

### Step 3: Transition

* Choose digit `d` at this position.
* Range: `0 → digits[pos]` if tight, else `0 → 9`.
* If `d == 7`, skip (invalid).
* Update:

  * `new_tight = tight and (d == digits[pos])`
  * `new_leading_zero = leading_zero and (d == 0)`

---

### Step 4: Base Case

If we processed all digits (`pos == len(digits)`):

* return `1` (valid number).





# Problem: Count how many numbers ≤ N do not contain digit 7.

In [1]:
def count_without_7_rec(n: int) -> int:
    digits = list(map(int, str(n)))

    def dfs(pos, tight, leading_zero):
        # Base case: processed all digits.
        if pos == len(digits):
            # To tell its a valid number.
            return 1

        limit = digits[pos] if tight else 9
        res = 0

        for d in range(0, limit + 1):
            if d == 7:
                continue  # skip invalid digit
            new_tight = tight and (d == limit)
            # How when the number is 325, when we are at pos = 0 and d = 3. the next values also should be a tight.
            new_leading_zero = leading_zero and (d == 0)
            # leading zero, not used in this sum. but usedful later. keeping here to fullfil the logic.
            res += dfs(pos + 1, new_tight, new_leading_zero)

        return res

    return dfs(0, True, True)


In [ ]:
count_without_7_rec(20)
# from 0 - 20 - there will be 19 numbers without having 7.

19

In [ ]:
def count_without_7_bottomup(n: int) -> int:
    digits = list(map(int, str(n)))
    L = len(digits)

    # dp[pos][tight][leading_zero]
    dp = [[[0]*2 for _ in range(2)] for _ in range(L+1)] # one extra row for base case.
    print(dp)

    # Base case
    for tight in range(2):
        for leading_zero in range(2):
            dp[L][tight][leading_zero] = 1  # setting all the last rows values to 1.
    print(dp)

    # [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[1, 1], [1, 1]]]
    # for n = 200
    # 3 - rows 2 cols 2 width

    for pos in range(L-1, -1, -1):
        for tight in range(2):
            for leading_zero in range(2):
                limit = digits[pos] if tight else 9
                res = 0
                for d in range(0, limit+1):
                    if d == 7:
                        continue
                    new_tight = tight and (d == limit)
                    new_leading_zero = leading_zero and (d == 0)
                    res += dp[pos+1][new_tight][new_leading_zero]
                dp[pos][tight][leading_zero] = res

    return dp[0][1][1]

# tc - o(pos * 2 * 2)
# sc - o(pos * 2 * 2)

In [7]:
count_without_7_bottomup(200)

[[[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]]]
[[[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]], [[1, 1], [1, 1]]]


163

In [4]:
def rotate_number(p: str): # p :int 

    for i in range(len(p)):
        if p[i] not in ['1', '0', '8']:
            print("here")
            print(p[i], i)
            return False
    if p==p[::-1]:
        return True
    else:
        print("here1")
        return False
    
rotate_number('818')

True

In [ ]:
import dask.dataframe as dd
import json
import hashlib

# Read partitioned parquet
df = dd.read_parquet("s3://ew1-scmp-text-anonymizer-prd-system-sagemaker/anon-sms-data")

# Drop unnecessary columns
df = df.drop(columns=["Message", "UrlArray"])

# Hash SenderId using SHA-256
def hash_sender(sender):
    if sender is None:
        return None
    return hashlib.sha256(str(sender).encode("utf-8")).hexdigest()

df["SenderId"] = df["SenderId"].map(hash_sender, meta=("SenderId", "str"))

# Process ExtractedFeatures: count lists, drop "text" & "trusted_urls"
def process_features(val):
    try:
        d = val if isinstance(val, dict) else json.loads(val)
    except Exception:
        return "{}"
    result = {}
    for k, v in d.items():
        if k in ["text", "trusted_urls"]:
            continue
        if isinstance(v, list):
            result[k] = len(v)
        else:
            result[k] = v
    return json.dumps(result)

df["ExtractedFeatures"] = df["ExtractedFeatures"].map(
    process_features, meta=("ExtractedFeatures", "str")
)

# Extra safety: ensure column is strictly string (no dicts left)
df["ExtractedFeatures"] = df["ExtractedFeatures"].astype(str)

# Ensure UrlApiReputations is stored as JSON string
df["UrlApiReputations"] = df["UrlApiReputations"].map(
    lambda x: json.dumps(x) if isinstance(x, (list, tuple, dict)) else str(x),
    meta=("UrlApiReputations", "str")
)

df["UrlApiReputations"] = df["UrlApiReputations"].astype(str)

# Write back as partitioned parquet
df.to_parquet(
    "s3://ew1-scmp-text-anonymizer-prd-system-sagemaker/anon-sms-data-cleaned/",
    engine="pyarrow",
    partition_on=["month", "CreatedDate"],
    overwrite=True
)
 